In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.read_csv('../stack-overflow-developer-survey-2022/survey_results_public.csv')
data = data.dropna(subset=['DevType'])

In [3]:
def job_cluster(x):
    # if student/other in multiple type answer, delete student/other
    if 'Student' in x:
        x = x.replace('Student;', '')
        x = x.replace(';Student', '')
    if 'Other (please specify):' in x:
        x = x.replace('Other (please specify):;', '')
        x = x.replace(';Other (please specify):', '')

    
    # order of processing is important (from cross-domain to specific)
    if 'Educator' in x:
        return 'Educator'
    elif any(word in x for word in ['manager', 'Executive']):
        return 'Manager'
    elif any(word in x for word in ['Data ', 'data', 'Marketing']):
        return 'Data'
    elif 'full-stack' in x:
        return 'Developer FS'
    elif 'game' in x:
        return 'Game Dev'
    elif any(word in x for word in ['front-end', 'Designer']):
        return 'Developer FE'
    elif 'back-end' in x:
        return 'Developer BE'
    elif 'Developer,' in x:
        return 'Testing/Apps/mobile'
    elif 'admin' in x:
        return 'Admin'
    elif any(word in x for word in ['DevOps', 'reliability']):
        return 'DevOps'
    elif any(word in x for word in ['Security', 'Blockchain', 'Cloud']):
        return 'Security/Cloud'
    elif any(word in x for word in ['researcher', 'Scientist']):
        return 'Researching'
    else:
        return x

data['TYPE'] = data['DevType'].apply(job_cluster)

In [4]:
def numerize(x):
    if x == 'Less than 1 year':
        return 0
    elif x == 'More than 50 years':
        return 51
    else:
        return x

In [5]:
devs = data[['TYPE', 'YearsCode', 'MainBranch', 'Country', 'ConvertedCompYearly', 'Employment']]
devs['YearsCode'] = devs['YearsCode'].fillna(0)
devs['YearsCode'] = devs['YearsCode'].apply(numerize)
devs.head(1)
devs.dropna(subset=['ConvertedCompYearly']).to_csv('../csv-data/sals.csv')

/tmp/ipykernel_110848/1562436368.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  devs['YearsCode'] = devs['YearsCode'].fillna(0)
/tmp/ipykernel_110848/1562436368.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  devs['YearsCode'] = devs['YearsCode'].apply(numerize)


In [6]:
sals = devs.groupby(['Country']).agg({'ConvertedCompYearly':np.mean, 'TYPE':stats.mode})
sals

/home/mabioo/.local/lib/python3.10/site-packages/pandas/core/groupby/groupby.py:1649: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  f = lambda x: func(x, *args, **kwargs)
/home/mabioo/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "


,ConvertedCompYearly,TYPE
Country,,
Afghanistan,33914.666667,"([Developer FS], [14])"
Albania,24895.148148,"([Developer FS], [20])"
Algeria,12732.888889,"([Developer FS], [10])"
Andorra,98003.125000,"([Developer BE], [3])"
Angola,6222.750000,"([Developer FS], [4])"
...,...,...
"Venezuela, Bolivarian Republic of...",17079.400000,"([Developer FS], [37])"
Viet Nam,38093.537634,"([Developer FS], [76])"
Yemen,17464.666667,"([Developer FS], [4])"


In [7]:
def splitter(x: str):
    return x.split('(')[0].rstrip()

def encode_multichoices(column):
    df = pd.DataFrame(data[column])
    df = df[column].str.get_dummies(sep=';')
    df = df.drop(['Other (Please specify):'], axis=1, errors='ignore')
    df = df.drop(['Other (please specify):'], axis=1, errors='ignore')
    df.columns = list(map(splitter, df.columns))  
    # df = df.add_prefix(f'{column}_')
    return df

In [8]:
code = encode_multichoices('DevType')
devs = pd.concat([code, devs], axis=1)

devs.to_csv('../csv-data/rudi.csv')